In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.math as tm
import numpy as np
import random
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.metrics import accuracy_score

In [2]:
x_train = np.array([[0, 0],
           [0, 1],
           [1, 0],
           [1, 1]])
y_train = np.array([[0],
           [1],
           [1],
           [0]])
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(4)

In [3]:
def convert2_zero_one(x):
    
    t = [tm.sigmoid(i) for i in x]
    
    return t

In [4]:
def rerange(x, r = 6.0):
    
    out_of_range = tf.cast(tm.greater(tm.abs(x), r), tf.float32)
    sign = tm.sign(x)
    
    return x * (1 - out_of_range) + sign * r * out_of_range

In [5]:
def cont_bern_log_norm(lam, l_lim=0.49, u_lim=0.51):
    '''
    computes the log normalizing constant of a continuous Bernoulli distribution in a numerically stable way.
    returns the log normalizing constant for lam in (0, l_lim) U (u_lim, 1) and a Taylor approximation in
    [l_lim, u_lim].
    cut_y below might appear useless, but it is important to not evaluate log_norm near 0.5 as tf.where evaluates
    both options, regardless of the value of the condition.
    '''
    
    cut_lam = tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), lam, l_lim * tf.ones_like(lam))
    log_norm = tm.log(tm.abs(2.0 * tm.atanh(1 - 2.0 * cut_lam))) - tm.log(tm.abs(1 - 2.0 * cut_lam))
    taylor = tm.log(2.0) + 4.0 / 3.0 * tm.pow(lam - 0.5, 2) + 104.0 / 45.0 * tm.pow(lam - 0.5, 4)
    return tf.where(tm.logical_or(tm.less(lam, l_lim), tm.greater(lam, u_lim)), log_norm, taylor)

In [44]:
# MLP model
class StochasticMLP(Model):
    
    def __init__(self, hidden_layer_sizes=[100], n_outputs=10):
        super(StochasticMLP, self).__init__()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.fc_layers = [Dense(layer_size) for layer_size in hidden_layer_sizes]
        self.output_layer = Dense(n_outputs)
    
    def call(self, x):
        
        #x = Flatten()(x)
        
        network = []
        
        for i, layer in enumerate(self.fc_layers):
            
            logits = layer(x)
            x = tfp.distributions.Bernoulli(logits=logits).sample()
            network.append(x)

        final_logits = self.output_layer(x) # initial the weight of output layer
            
        return network
    
    def target_log_prob(self, x, h, y):
        
        h_current = convert2_zero_one([tf.cast(h_i, dtype=tf.float32) for h_i in h])
        h_previous = [x] + h_current[:-1]
    
        nlog_prob = 0. # negative log probability
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current, h_previous, self.fc_layers)):
            
            logits = layer(pv)
            
            ce = tf.nn.sigmoid_cross_entropy_with_logits(
                labels=cv, logits=logits)
            
            ce += cont_bern_log_norm(tf.nn.sigmoid(logits))
            
            nlog_prob += tf.reduce_sum(ce, axis = -1)
        
        fce = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.cast(y, tf.float32), logits=self.output_layer(h_current[-1]))
        nlog_prob += tf.reduce_sum(fce, axis = -1)
            
        return -1 * nlog_prob

    def target_log_prob2(self, x, h, y):

        h_current = convert2_zero_one(tf.split(h, self.hidden_layer_sizes, axis = 1))
        h_previous = [x] + h_current[:-1]
        
        nlog_prob = 0.
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current, h_previous, self.fc_layers)):
            
            logits = layer(pv)
            ce = tf.nn.sigmoid_cross_entropy_with_logits(
                labels=cv, logits=logits)
            
            ce += cont_bern_log_norm(tf.nn.sigmoid(logits))
            
            nlog_prob += tf.reduce_sum(ce, axis = -1)
        
        fce = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.cast(y, tf.float32), logits=self.output_layer(h_current[-1]))
        nlog_prob += tf.reduce_sum(fce, axis = -1)
            
        return -1 * nlog_prob
    
    def generate_hmc_kernel(self, x, y, step_size = pow(1000, -1/4)):
        
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = lambda v: self.target_log_prob2(x, v, y),
            num_leapfrog_steps = 2,
            step_size = step_size),
            num_adaptation_steps=int(100 * 0.8))
        
        return adaptive_hmc
    
    # new proposing-state method with HamiltonianMonteCarlo
    def propose_new_state_hamiltonian(self, x, h, y, hmc_kernel):
    
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        h_current = tf.concat(h_current, axis = 1)

        # run the chain (with burn-in)
        num_burnin_steps = 0
        num_results = 1

        samples = tfp.mcmc.sample_chain(
            num_results = num_results,
            num_burnin_steps = num_burnin_steps,
            current_state = h_current, # may need to be reshaped
            kernel = hmc_kernel,
            trace_fn = None,
            return_final_kernel_results = True)
        
        new_step_size = samples[2].new_step_size.numpy()
        ker_new = self.generate_hmc_kernel(x, y, new_step_size)
        
        #h_state = rerange(samples[0][0])
        h_state = samples[0][0]
        h_new = tf.split(h_state, self.hidden_layer_sizes, axis = 1)   

        return(h_new, ker_new)
    
    def update_weights(self, x, h, y, lr = 0.1):
        
        optimizer = tf.keras.optimizers.SGD(learning_rate = lr)
        with tf.GradientTape() as tape:
            loss = -1 * tf.reduce_mean(self.target_log_prob(x, h, y))
        
        grads = tape.gradient(loss, self.trainable_weights)
        optimizer.apply_gradients(zip(grads, self.trainable_weights))

    def get_predictions(self, x):

        logits = 0.0
        for layer in self.fc_layers:
            logits = layer(x)
            x = tm.sigmoid(logits)
        
        logits = self.output_layer(x)
        probs = tm.sigmoid(logits)
        #print(probs)
        labels = tf.cast(tm.greater(probs, 0.5), tf.int32)

        return labels       

In [7]:
model = StochasticMLP(hidden_layer_sizes = [2], n_outputs=1)

In [8]:
network = [model.call(images) for images, labels in train_ds]

In [9]:
# Set weight
w_0 = np.array([[1, -1], [1, -1]], dtype = "float32")
b_0 = np.array([-0.5, 1], dtype = "float32")
l_0 = [w_0, b_0]

w_1 = np.array([[1], [1]], dtype = "float32")
b_1 = np.array([-1], dtype = "float32")
l_1 = [w_1, b_1]

model.fc_layers[0].set_weights(l_0)
model.output_layer.set_weights(l_1)

In [ ]:
network = [model.call(images) for images, labels in train_ds]
network

In [ ]:
kernels = [model.generate_hmc_kernel(images, labels) for images, labels in train_ds]

In [ ]:
burnin = 1000
step_sizes = []
for i in range(burnin):
    
    if(i % 100 == 0): print("Step %d" % i)
    
    network_new = []
    kernels_new = []
    
    res = [model.propose_new_state_hamiltonian(x, net, y, ker) 
               for (x, y), net, ker in zip(train_ds, network, kernels)]
    
    network_new, kernels_new = zip(*res)
         
    network = network_new
    kernels = kernels_new

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(step_sizes)), step_sizes)
plt.show()

In [ ]:
print(network)

In [ ]:
epochs = 500

start_time = time.time()
for epoch in range(epochs):
    
    loss = 0.0
    acc = 0.0
    for bs, (images, labels) in enumerate(train_ds):
        
        # only one mini-batch
        model.update_weights(images, network[bs], labels, 0.1)
        #network_new = [model.propose_new_state_hamiltonian(images, net, labels) for (images, labels), net in 
        #               zip(train_ds, network)]
        
        network_new = []
        #kernels_new = []
        for net, hmc_kernel in zip(network, kernels):
            net_current = net
            net_current = [tf.cast(net_i, dtype=tf.float32) for net_i in net_current]
            net_current = tf.concat(net_current, axis = 1)
        
            num_results = 1
            num_burnin_steps = 0

            samples = tfp.mcmc.sample_chain(
                num_results = num_results,
                num_burnin_steps = num_burnin_steps,
                current_state = net_current, # may need to be reshaped
                kernel = hmc_kernel,
                trace_fn = None,
                return_final_kernel_results = True)
            
            new_state = rerange(samples[0][0])
            net_new = tf.split(new_state, [2], axis = 1)   
            network_new.append(net_new)

        network = network_new
        
        loss += -1 * tf.reduce_mean(model.target_log_prob(images, network[bs], labels))
    
    preds = [model.get_predictions(images) for images, labels in train_ds]
    #print(preds)
    train_acc = accuracy_score(np.array(preds[0]), y_train)
    print("Epoch %d/%d: - %.4fs/step - loss: %.4f - accuracy: %.4f" 
          % (epoch + 1, epochs, (time.time() - start_time) / (epoch + 1), loss, train_acc))

In [ ]:
print(model.trainable_weights)

In [54]:
model2 = StochasticMLP(hidden_layer_sizes = [30], n_outputs=1)
network2 = [model2.call(images) for images, labels in train_ds]
kernels2 = [model2.generate_hmc_kernel(images, labels) for images, labels in train_ds]

In [55]:
print(network2)

[[<tf.Tensor: shape=(4, 30), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 1]], dtype=int32)>]]


In [56]:
burnin = 1000
step_sizes2 = []
for i in range(burnin):
    
    if(i % 100 == 0): print("Step %d" % i)
    
    network2_new = []
    kernels2_new = []
    
    res = [model2.propose_new_state_hamiltonian(x, net, y, ker) 
               for (x, y), net, ker in zip(train_ds, network2, kernels2)]
    
    network2_new, kernels2_new = zip(*res)
         
    network2 = network2_new
    kernels2 = kernels2_new

    

Step 0
Step 100
Step 200
Step 300
Step 400
Step 500
Step 600
Step 700
Step 800
Step 900


In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(step_sizes2)), step_sizes2)
plt.show()
pass

In [57]:
print(network2[0])

[<tf.Tensor: shape=(4, 30), dtype=float32, numpy=
array([[  6873.3066 ,   4639.7314 ,  -9557.695  ,   7499.932  ,
          1029.634  ,   8471.064  ,   3020.163  ,   5515.167  ,
         -3697.842  ,  19789.023  , -11206.564  ,   -534.3258 ,
        -10604.851  , -18439.115  ,  10346.445  ,   7199.4604 ,
         -3496.1953 ,  -1334.2058 ,   -872.68164,  -9802.813  ,
          3506.5645 ,  13825.773  , -10822.824  ,  29016.238  ,
        -10671.174  ,   5360.4395 ,   8623.091  ,   9702.884  ,
           867.2149 ,   2368.781  ],
       [ -3470.1    ,  20320.27   ,  12667.398  ,  -1427.4838 ,
          6150.4023 ,   1070.3035 ,  -5632.833  , -11669.932  ,
          9812.163  ,  17764.393  ,    342.58014,  -2962.7686 ,
         18805.047  ,   2574.5017 ,   -743.58295, -11773.479  ,
         -3690.029  ,    350.1704 ,   5858.4277 ,   7959.5176 ,
          9229.361  ,  14574.923  ,   7276.674  ,   2940.6167 ,
          -940.9846 ,  -6420.539  ,  11179.929  ,  10083.695  ,
        -17381.07

In [58]:
epochs = 1500

start_time = time.time()
for epoch in range(epochs):
    
    loss = 0.0
    acc = 0.0
    for bs, (images, labels) in enumerate(train_ds):
        
        # only one mini-batch
        model2.update_weights(images, network2[bs], labels, 0.1)
        #network_new = [model.propose_new_state_hamiltonian(images, net, labels) for (images, labels), net in 
        #               zip(train_ds, network)]
        
        network_new2 = []
        for (images, labels), net, hmc_kernel in zip(train_ds, network2, kernels2):
            net_current = net
            net_current = [tf.cast(net_i, dtype=tf.float32) for net_i in net_current]
            net_current = tf.concat(net_current, axis = 1)
        
            num_results = 1
            num_burnin_steps = 0

            samples = tfp.mcmc.sample_chain(
                num_results = num_results,
                num_burnin_steps = num_burnin_steps,
                current_state = net_current, # may need to be reshaped
                kernel = hmc_kernel,
                trace_fn = None,
                return_final_kernel_results = True)
            
            #new_state = rerange(samples[0][0])
            new_state = samples[0][0]
            net_new = tf.split(new_state, [30], axis = 1)   
            network_new2.append(net_new)
            
        network2 = network_new2
        
        loss += -1 * tf.reduce_mean(model2.target_log_prob(images, network2[bs], labels))
    
    preds = [model2.get_predictions(images) for images, labels in train_ds]
    train_acc = accuracy_score(np.array(preds[0]), y_train)
    print("Epoch %d/%d: - %.4fs/step - loss: %.4f - accuracy: %.4f" 
          % (epoch + 1, epochs, (time.time() - start_time) / (epoch + 1), loss, train_acc))

Epoch 1/1500: - 0.1383s/step - loss: 41.7115 - accuracy: 0.5000
Epoch 2/1500: - 0.1087s/step - loss: 41.1930 - accuracy: 0.5000
Epoch 3/1500: - 0.1022s/step - loss: 40.7914 - accuracy: 0.5000
Epoch 4/1500: - 0.0991s/step - loss: 40.4091 - accuracy: 0.5000
Epoch 5/1500: - 0.1087s/step - loss: 40.2314 - accuracy: 0.5000
Epoch 6/1500: - 0.1039s/step - loss: 40.1298 - accuracy: 0.5000
Epoch 7/1500: - 0.1108s/step - loss: 39.9304 - accuracy: 0.5000
Epoch 8/1500: - 0.1148s/step - loss: 39.6901 - accuracy: 0.5000
Epoch 9/1500: - 0.1144s/step - loss: 39.9190 - accuracy: 0.5000
Epoch 10/1500: - 0.1120s/step - loss: 39.6105 - accuracy: 0.5000
Epoch 11/1500: - 0.1117s/step - loss: 39.1741 - accuracy: 0.5000
Epoch 12/1500: - 0.1089s/step - loss: 38.9496 - accuracy: 0.5000
Epoch 13/1500: - 0.1246s/step - loss: 38.8287 - accuracy: 0.5000
Epoch 14/1500: - 0.1226s/step - loss: 38.6112 - accuracy: 0.5000
Epoch 15/1500: - 0.1197s/step - loss: 38.5381 - accuracy: 0.5000
Epoch 16/1500: - 0.1191s/step - lo

Epoch 127/1500: - 0.0959s/step - loss: 35.8034 - accuracy: 1.0000
Epoch 128/1500: - 0.0966s/step - loss: 35.7336 - accuracy: 0.7500
Epoch 129/1500: - 0.0968s/step - loss: 35.8277 - accuracy: 0.7500
Epoch 130/1500: - 0.0966s/step - loss: 35.6115 - accuracy: 0.7500
Epoch 131/1500: - 0.0966s/step - loss: 35.9375 - accuracy: 0.7500
Epoch 132/1500: - 0.0967s/step - loss: 36.0668 - accuracy: 0.5000
Epoch 133/1500: - 0.0969s/step - loss: 36.0605 - accuracy: 0.5000
Epoch 134/1500: - 0.0968s/step - loss: 35.9715 - accuracy: 0.5000
Epoch 135/1500: - 0.0967s/step - loss: 35.9652 - accuracy: 0.5000
Epoch 136/1500: - 0.0968s/step - loss: 35.9537 - accuracy: 0.5000
Epoch 137/1500: - 0.0966s/step - loss: 35.7596 - accuracy: 0.5000
Epoch 138/1500: - 0.0971s/step - loss: 35.7724 - accuracy: 1.0000
Epoch 139/1500: - 0.0973s/step - loss: 35.7186 - accuracy: 0.5000
Epoch 140/1500: - 0.0972s/step - loss: 36.1283 - accuracy: 0.5000
Epoch 141/1500: - 0.0971s/step - loss: 35.9130 - accuracy: 1.0000
Epoch 142/

Epoch 253/1500: - 0.1059s/step - loss: 35.5004 - accuracy: 0.7500
Epoch 254/1500: - 0.1059s/step - loss: 35.4454 - accuracy: 0.7500
Epoch 255/1500: - 0.1058s/step - loss: 35.3941 - accuracy: 0.7500
Epoch 256/1500: - 0.1057s/step - loss: 35.0815 - accuracy: 0.7500
Epoch 257/1500: - 0.1057s/step - loss: 35.0419 - accuracy: 0.7500
Epoch 258/1500: - 0.1060s/step - loss: 34.7636 - accuracy: 0.7500
Epoch 259/1500: - 0.1059s/step - loss: 34.9956 - accuracy: 0.7500
Epoch 260/1500: - 0.1058s/step - loss: 34.9632 - accuracy: 0.7500
Epoch 261/1500: - 0.1057s/step - loss: 34.9325 - accuracy: 0.7500
Epoch 262/1500: - 0.1058s/step - loss: 34.6562 - accuracy: 0.7500
Epoch 263/1500: - 0.1060s/step - loss: 34.6737 - accuracy: 0.7500
Epoch 264/1500: - 0.1059s/step - loss: 34.6447 - accuracy: 0.7500
Epoch 265/1500: - 0.1057s/step - loss: 34.5476 - accuracy: 0.7500
Epoch 266/1500: - 0.1059s/step - loss: 34.5154 - accuracy: 0.7500
Epoch 267/1500: - 0.1061s/step - loss: 34.7511 - accuracy: 0.7500
Epoch 268/

Epoch 379/1500: - 0.1010s/step - loss: 34.0531 - accuracy: 0.7500
Epoch 380/1500: - 0.1009s/step - loss: 34.0800 - accuracy: 0.7500
Epoch 381/1500: - 0.1009s/step - loss: 34.2236 - accuracy: 0.7500
Epoch 382/1500: - 0.1008s/step - loss: 34.1903 - accuracy: 0.7500
Epoch 383/1500: - 0.1007s/step - loss: 34.5437 - accuracy: 0.7500
Epoch 384/1500: - 0.1006s/step - loss: 34.5107 - accuracy: 0.7500
Epoch 385/1500: - 0.1006s/step - loss: 34.4800 - accuracy: 0.7500
Epoch 386/1500: - 0.1005s/step - loss: 34.4515 - accuracy: 0.7500
Epoch 387/1500: - 0.1005s/step - loss: 34.7248 - accuracy: 0.7500
Epoch 388/1500: - 0.1004s/step - loss: 34.6709 - accuracy: 0.7500
Epoch 389/1500: - 0.1004s/step - loss: 34.6212 - accuracy: 0.7500
Epoch 390/1500: - 0.1003s/step - loss: 34.6543 - accuracy: 0.7500
Epoch 391/1500: - 0.1002s/step - loss: 34.5287 - accuracy: 0.7500
Epoch 392/1500: - 0.1002s/step - loss: 34.4904 - accuracy: 0.7500
Epoch 393/1500: - 0.1001s/step - loss: 35.0309 - accuracy: 0.7500
Epoch 394/

Epoch 504/1500: - 0.1017s/step - loss: 34.5846 - accuracy: 0.7500
Epoch 505/1500: - 0.1017s/step - loss: 34.5999 - accuracy: 0.7500
Epoch 506/1500: - 0.1016s/step - loss: 34.5153 - accuracy: 0.7500
Epoch 507/1500: - 0.1016s/step - loss: 34.4735 - accuracy: 0.7500
Epoch 508/1500: - 0.1016s/step - loss: 34.4345 - accuracy: 0.7500
Epoch 509/1500: - 0.1018s/step - loss: 34.3981 - accuracy: 0.7500
Epoch 510/1500: - 0.1018s/step - loss: 34.3641 - accuracy: 0.7500
Epoch 511/1500: - 0.1017s/step - loss: 34.7540 - accuracy: 0.7500
Epoch 512/1500: - 0.1016s/step - loss: 34.6508 - accuracy: 0.7500
Epoch 513/1500: - 0.1018s/step - loss: 34.3778 - accuracy: 0.7500
Epoch 514/1500: - 0.1019s/step - loss: 34.4404 - accuracy: 0.7500
Epoch 515/1500: - 0.1018s/step - loss: 34.3349 - accuracy: 0.7500
Epoch 516/1500: - 0.1017s/step - loss: 34.3112 - accuracy: 0.7500
Epoch 517/1500: - 0.1018s/step - loss: 34.2886 - accuracy: 0.7500
Epoch 518/1500: - 0.1019s/step - loss: 34.2672 - accuracy: 0.7500
Epoch 519/

...

Epoch 882/1500: - 0.1075s/step - loss: 34.5440 - accuracy: 1.0000
Epoch 883/1500: - 0.1074s/step - loss: 34.6811 - accuracy: 1.0000
Epoch 884/1500: - 0.1074s/step - loss: 34.4486 - accuracy: 1.0000
Epoch 885/1500: - 0.1073s/step - loss: 34.4051 - accuracy: 1.0000
Epoch 886/1500: - 0.1074s/step - loss: 34.3750 - accuracy: 1.0000
Epoch 887/1500: - 0.1074s/step - loss: 34.3377 - accuracy: 1.0000
Epoch 888/1500: - 0.1074s/step - loss: 34.3031 - accuracy: 1.0000
Epoch 889/1500: - 0.1074s/step - loss: 34.2710 - accuracy: 1.0000
Epoch 890/1500: - 0.1074s/step - loss: 34.2411 - accuracy: 1.0000
Epoch 891/1500: - 0.1074s/step - loss: 34.4344 - accuracy: 1.0000
Epoch 892/1500: - 0.1074s/step - loss: 34.3979 - accuracy: 1.0000
Epoch 893/1500: - 0.1073s/step - loss: 34.3216 - accuracy: 1.0000
Epoch 894/1500: - 0.1074s/step - loss: 34.6713 - accuracy: 1.0000
Epoch 895/1500: - 0.1074s/step - loss: 34.6154 - accuracy: 1.0000
Epoch 896/1500: - 0.1074s/step - loss: 34.5636 - accuracy: 1.0000
Epoch 897/

Epoch 1009/1500: - 0.1081s/step - loss: 34.3260 - accuracy: 0.7500
Epoch 1010/1500: - 0.1081s/step - loss: 33.9850 - accuracy: 0.7500
Epoch 1011/1500: - 0.1082s/step - loss: 34.2989 - accuracy: 0.7500
Epoch 1012/1500: - 0.1082s/step - loss: 34.2624 - accuracy: 0.7500
Epoch 1013/1500: - 0.1082s/step - loss: 34.1294 - accuracy: 0.7500
Epoch 1014/1500: - 0.1082s/step - loss: 33.9022 - accuracy: 0.7500
Epoch 1015/1500: - 0.1083s/step - loss: 33.9486 - accuracy: 0.7500
Epoch 1016/1500: - 0.1083s/step - loss: 33.7846 - accuracy: 0.7500
Epoch 1017/1500: - 0.1083s/step - loss: 33.7730 - accuracy: 0.7500
Epoch 1018/1500: - 0.1084s/step - loss: 33.7618 - accuracy: 0.7500
Epoch 1019/1500: - 0.1084s/step - loss: 33.7509 - accuracy: 0.7500
Epoch 1020/1500: - 0.1084s/step - loss: 33.7404 - accuracy: 0.7500
Epoch 1021/1500: - 0.1085s/step - loss: 33.9051 - accuracy: 0.7500
Epoch 1022/1500: - 0.1085s/step - loss: 33.8916 - accuracy: 0.7500
Epoch 1023/1500: - 0.1085s/step - loss: 33.8786 - accuracy: 0.

Epoch 1132/1500: - 0.1098s/step - loss: 33.9628 - accuracy: 0.7500
Epoch 1133/1500: - 0.1097s/step - loss: 33.9348 - accuracy: 0.7500
Epoch 1134/1500: - 0.1097s/step - loss: 34.1151 - accuracy: 0.7500
Epoch 1135/1500: - 0.1097s/step - loss: 34.2304 - accuracy: 0.7500
Epoch 1136/1500: - 0.1097s/step - loss: 34.1867 - accuracy: 0.7500
Epoch 1137/1500: - 0.1096s/step - loss: 34.0325 - accuracy: 0.7500
Epoch 1138/1500: - 0.1096s/step - loss: 34.1147 - accuracy: 0.7500
Epoch 1139/1500: - 0.1096s/step - loss: 33.9324 - accuracy: 0.7500
Epoch 1140/1500: - 0.1096s/step - loss: 33.8058 - accuracy: 0.7500
Epoch 1141/1500: - 0.1096s/step - loss: 33.7864 - accuracy: 0.7500
Epoch 1142/1500: - 0.1095s/step - loss: 33.7683 - accuracy: 0.7500
Epoch 1143/1500: - 0.1095s/step - loss: 33.8748 - accuracy: 0.7500
Epoch 1144/1500: - 0.1095s/step - loss: 33.8570 - accuracy: 0.7500
Epoch 1145/1500: - 0.1095s/step - loss: 33.8401 - accuracy: 0.7500
Epoch 1146/1500: - 0.1094s/step - loss: 33.8241 - accuracy: 0.

Epoch 1257/1500: - 0.1099s/step - loss: 33.2127 - accuracy: 0.7500
Epoch 1258/1500: - 0.1099s/step - loss: 33.4531 - accuracy: 0.7500
Epoch 1259/1500: - 0.1099s/step - loss: 33.6308 - accuracy: 0.7500
Epoch 1260/1500: - 0.1100s/step - loss: 33.6112 - accuracy: 0.7500
Epoch 1261/1500: - 0.1100s/step - loss: 33.9539 - accuracy: 0.7500
Epoch 1262/1500: - 0.1100s/step - loss: 33.9181 - accuracy: 0.7500
Epoch 1263/1500: - 0.1099s/step - loss: 33.8846 - accuracy: 0.7500
Epoch 1264/1500: - 0.1099s/step - loss: 33.8533 - accuracy: 0.7500
Epoch 1265/1500: - 0.1100s/step - loss: 33.8241 - accuracy: 0.7500
Epoch 1266/1500: - 0.1100s/step - loss: 34.1855 - accuracy: 0.7500
Epoch 1267/1500: - 0.1100s/step - loss: 33.7714 - accuracy: 0.7500
Epoch 1268/1500: - 0.1100s/step - loss: 33.8691 - accuracy: 0.7500
Epoch 1269/1500: - 0.1101s/step - loss: 33.8387 - accuracy: 0.7500
Epoch 1270/1500: - 0.1101s/step - loss: 33.4810 - accuracy: 0.7500
Epoch 1271/1500: - 0.1100s/step - loss: 33.4690 - accuracy: 0.

Epoch 1382/1500: - 0.1100s/step - loss: 33.3234 - accuracy: 0.7500
Epoch 1383/1500: - 0.1100s/step - loss: 33.2971 - accuracy: 0.7500
Epoch 1384/1500: - 0.1100s/step - loss: 33.2729 - accuracy: 0.7500
Epoch 1385/1500: - 0.1100s/step - loss: 33.2506 - accuracy: 0.7500
Epoch 1386/1500: - 0.1100s/step - loss: 33.2301 - accuracy: 0.7500
Epoch 1387/1500: - 0.1099s/step - loss: 33.4629 - accuracy: 0.7500
Epoch 1388/1500: - 0.1100s/step - loss: 33.2315 - accuracy: 0.7500
Epoch 1389/1500: - 0.1100s/step - loss: 33.2054 - accuracy: 0.7500
Epoch 1390/1500: - 0.1100s/step - loss: 33.1409 - accuracy: 0.7500
Epoch 1391/1500: - 0.1099s/step - loss: 33.1224 - accuracy: 0.7500
Epoch 1392/1500: - 0.1099s/step - loss: 33.1056 - accuracy: 0.7500
Epoch 1393/1500: - 0.1100s/step - loss: 33.0901 - accuracy: 0.7500
Epoch 1394/1500: - 0.1100s/step - loss: 33.0760 - accuracy: 0.7500
Epoch 1395/1500: - 0.1099s/step - loss: 33.4073 - accuracy: 0.7500
Epoch 1396/1500: - 0.1099s/step - loss: 33.1170 - accuracy: 0.